Dans cette dernière partie, nous allons créer des cartes intéractives pour recommander à nos clients des endroits ou partir en vacances en fonction de conditions météo et critères de notation d'hotels

In [2]:
import requests
import pandas as pd
import json
import os
from collections import Counter
import numpy as np
from google.colab import drive
import plotly.graph_objects as go
import plotly.express as px

In [3]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
cities = ["Mont Saint Michel","St Malo","Bayeux","Le Havre","Rouen","Paris","Amiens","Lille","Strasbourg",
          "Chateau du Haut Koenigsbourg","Colmar","Eguisheim","Besancon","Dijon","Annecy","Grenoble","Lyon",
          "Gorges du Verdon","Bormes les Mimosas","Cassis","Marseille","Aix en Provence","Avignon","Uzes",
          "Nimes","Aigues Mortes","Saintes Maries de la mer","Collioure","Carcassonne","Ariege","Toulouse",
          "Montauban","Biarritz","Bayonne","La Rochelle"]

In [5]:
kayak_flatten = pd.read_csv('/content/gdrive/MyDrive/datasets/kayak/flatten_weather.csv')
kayak = pd.read_csv("/content/gdrive/MyDrive/datasets/kayak/kayak2_final.csv")
df_total = pd.read_csv("/content/gdrive/MyDrive/datasets/kayak/best_hotels.csv")
df_flatten_final = pd.read_csv('/content/gdrive/MyDrive/datasets/kayak/df_flatten_final2.csv')

In [6]:
df_weather_flat = kayak_flatten.copy()
df_weather = kayak.copy()
df_hotels = df_total.copy()
df_top5 = df_flatten_final.copy()

### Weather maps

In [7]:
df_weather_flat['name'] = df_weather_flat['name'].apply(lambda x:x.replace(' ','-'))

In [8]:
df_weather_flat.head()

,city_id,name,latitude,longitude,days,temperature,mean_temp,proba_rain,mean_rain,humidity,...,pressure,mean_pressure,weather,mode_weather,score,date,mode_rain,mode_clouds,mode_clear,rank
0,29,Ariege,42.945537,1.406554,1,0,4,84,76.0,94,...,1016,1014,Snow,Rain,24.21,"11/18/2022, 13:07:14",4,1,0,1
1,29,Ariege,42.945537,1.406554,2,4,4,76,76.0,59,...,1020,1014,Rain,Rain,24.21,"11/18/2022, 13:07:14",4,1,0,1
2,29,Ariege,42.945537,1.406554,3,7,4,100,76.0,72,...,1007,1014,Rain,Rain,24.21,"11/18/2022, 13:07:14",4,1,0,1
3,29,Ariege,42.945537,1.406554,4,-3,4,100,76.0,96,...,1015,1014,Snow,Rain,24.21,"11/18/2022, 13:07:14",4,1,0,1
4,29,Ariege,42.945537,1.406554,5,5,4,98,76.0,84,...,1013,1014,Rain,Rain,24.21,"11/18/2022, 13:07:14",4,1,0,1


Voici une première map qui affiche tous les 35 villes que nous avions préalablement choisi avec les critères de température, de taux d'humidité, et de probabilité de pleuvoir

In [9]:
import plotly.express as px


fig = px.scatter_mapbox(
    df_weather[:35],
    lat='latitude',
    lon='longitude',
    color='mean_temp',
    size='rank',
    color_continuous_scale=px.colors.sequential.Bluered,
    range_color = [min(df_weather[:35]['mean_temp']), max(df_weather[:35]['mean_temp'])],
    size_max=35,
    zoom=4.7,
    hover_data={
        'latitude': False,
        'longitude': False,
        'proba_rain': True,
        'humidity': True,
        'temperature': True,
        'rank': False,
        },
    mapbox_style="carto-positron")

fig.update_layout(
    width = 900,
    height = 600,
    title_x=0.5,
    title_text='Carte des 35 villes de départs')

fig.show()

In [10]:
df_weather_flat.head()

,city_id,name,latitude,longitude,days,temperature,mean_temp,proba_rain,mean_rain,humidity,...,pressure,mean_pressure,weather,mode_weather,score,date,mode_rain,mode_clouds,mode_clear,rank
0,29,Ariege,42.945537,1.406554,1,0,4,84,76.0,94,...,1016,1014,Snow,Rain,24.21,"11/18/2022, 13:07:14",4,1,0,1
1,29,Ariege,42.945537,1.406554,2,4,4,76,76.0,59,...,1020,1014,Rain,Rain,24.21,"11/18/2022, 13:07:14",4,1,0,1
2,29,Ariege,42.945537,1.406554,3,7,4,100,76.0,72,...,1007,1014,Rain,Rain,24.21,"11/18/2022, 13:07:14",4,1,0,1
3,29,Ariege,42.945537,1.406554,4,-3,4,100,76.0,96,...,1015,1014,Snow,Rain,24.21,"11/18/2022, 13:07:14",4,1,0,1
4,29,Ariege,42.945537,1.406554,5,5,4,98,76.0,84,...,1013,1014,Rain,Rain,24.21,"11/18/2022, 13:07:14",4,1,0,1


Voici une carte intéractive des 5 meilleures destinations en fonction des jours de la semaine et de conditions météo

In [17]:
def plot_weather(df, size):
  fig = px.scatter_mapbox(
      df[:35],
      lat='latitude',
      lon='longitude',
      color='temperature',
      size=size,
      color_continuous_scale=px.colors.sequential.Bluered,
      size_max=35,
      zoom=4.7,
      range_color=[min(df[:35]['temperature']), max(df[:35]['temperature'])],
      hover_name='name',
      hover_data={
          'latitude': False,
          'longitude': False,
          'days': False,
          'proba_rain': True,
          'humidity': True,
          'temperature': True,
          'rank': True,
          },
      animation_frame='days',
      labels={'temperature': 'Temperature', 'proba_rain': 'Probability of raining', 'humidity': 'Humidity'}
  )

  fig.update_layout(
      width=1100,
      height=800,
      title_x=0.5,
      title_text='The 5 cities with the best weather over the next 7 days')

  fig.update_layout(mapbox_style='open-street-map')

  fig.show(renderer="colab")

In [18]:
plot_weather(df_weather_flat, size='humidity')

Pour cette partie, nous allons créer plusieurs fonctions (une qui trie par rapport à une colonne donnée, une autre qui normalise les données pour afficher des points de différentes tailles, 

In [19]:
new_df = df_weather
new_df = new_df.drop(['days', 'temperature','proba_rain', 'humidity','wind_speed', 'pressure', 'weather', 'rank'],axis=1)
new_df['mode_weather_num'] = new_df['mode_weather'].map({'Clear':1,'Clouds':2,'Rain':3})
new_df.columns

Index(['city_id', 'name', 'latitude', 'longitude', 'mean_temp', 'mean_rain',
       'mean_humidity', 'mean_wind_speed', 'mean_pressure', 'mode_weather',
       'score', 'date', 'mode_rain', 'mode_clouds', 'mode_clear',
       'mode_weather_num'],
      dtype='object')

In [20]:
from datetime import timedelta

new_df['date'] = pd.to_datetime(new_df['date'])
new_df['end_date'] = new_df['date'] + timedelta(days=6)

In [21]:
def sort_by_col(df, col, top):
  if col == 'mean_temp' or col == 'mean_pressure' or col == 'score':
    return df.sort_values(col, ascending=False)[:top]
  else:
    return df.sort_values(col, ascending=True)[:top]

In [22]:
def get_normalization(df, cols, standard_scaler=False, min_max_scaler=False):
  if standard_scaler:
    for col in cols:
      df[col] = df[col].apply(lambda x: ((x - df[col].mean()) / df[col].std()))
  if min_max_scaler:
    for col in cols:
      df[col] = df[col].apply(lambda x: ((x - df[col].min()) / (df[col].max() - df[col].min())))
  return df

In [23]:
cols = ['score', 'mean_temp', 'mean_rain', 'mean_humidity', 'mean_wind_speed', 'mean_pressure', 'mode_weather_num']

df_normalized = get_normalization(new_df, cols, min_max_scaler=True)
df_normalized.head()

,city_id,name,latitude,longitude,mean_temp,mean_rain,mean_humidity,mean_wind_speed,mean_pressure,mode_weather,score,date,mode_rain,mode_clouds,mode_clear,mode_weather_num,end_date
0,29,Ariege,42.945537,1.406554,0.1,0.758065,0.823529,0.2,1.000000,Rain,0.469462,2022-11-18 13:07:14,4,1,0,1.0,2022-11-24 13:07:14
1,9,Chateau du Haut Koenigsbourg,48.249523,7.345492,0.0,0.725806,0.764706,0.1,0.777778,Rain,0.390787,2022-11-18 13:07:14,5,2,0,1.0,2022-11-24 13:07:14
2,7,Lille,50.636565,3.063528,0.2,0.645161,0.764706,0.5,0.222222,Rain,0.502070,2022-11-18 13:07:14,7,0,0,1.0,2022-11-24 13:07:14
3,13,Dijon,47.321581,5.041470,0.2,0.693548,0.794118,0.4,0.888889,Rain,0.488095,2022-11-18 13:07:14,6,1,0,1.0,2022-11-24 13:07:14
4,14,Annecy,45.899235,6.128885,0.3,0.580645,1.000000,0.0,1.000000,Rain,0.429607,2022-11-18 13:07:14,5,1,0,1.0,2022-11-24 13:07:14


In [24]:
mapbox_access_token = 'pk.eyJ1Ijoib3AzbjVlZCIsImEiOiJjbDllYjl6bGswaG9uM3NsOW0zaGJ4ZHVrIn0.Us-gSPz0QgMbKbPqGkDtjg'

In [25]:
def create_dicts(df, top=None):

    dicts = {'df' : [sort_by_col(df=df, col='score', top=top),
                      sort_by_col(df=df, col='mean_temp', top=top),
                      sort_by_col(df=df, col='mean_humidity', top=top),
                      sort_by_col(df=df, col='mean_wind_speed', top=top),
                      sort_by_col(df=df, col='mode_weather_num', top=top),
                      sort_by_col(df=df, col='mean_pressure', top=top),
                      sort_by_col(df=df, col='mean_rain', top=top)],
            
            'df_normalized' : [sort_by_col(df=get_normalization(df, cols, min_max_scaler=True), col='score', top=top),
                                  sort_by_col(df=get_normalization(df, cols, min_max_scaler=True), col='mean_temp', top=top),
                                  sort_by_col(df=get_normalization(df, cols, min_max_scaler=True), col='mean_humidity', top=top),
                                  sort_by_col(df=get_normalization(df, cols, min_max_scaler=True), col='mean_wind_speed', top=top),
                                  sort_by_col(df=get_normalization(df, cols, min_max_scaler=True), col='mode_weather_num', top=top),
                                  sort_by_col(df=get_normalization(df, cols, min_max_scaler=True), col='mean_pressure', top=top),
                                  sort_by_col(df=get_normalization(df, cols, min_max_scaler=True), col='mean_rain', top=top)],
            
            'criterion' : ['score', 'mean_temp', 'mean_humidity', 'mean_wind_speed', 'mode_weather_num', 'mean_pressure', 'mean_rain'],
            
            'title' : ["Meilleur destination en fonction du score global",
                      "Meilleur destination en fonction de la temperture moyenne(°C)",
                      "Meilleur destination en fonction du taux d'humidité (%)",
                      "Meilleur destination en fonction de la vitesse du vent(noeuds)",
                      "Meilleure destination en fonction du ciel",
                      "Meilleure destination en fonction de la pression atmospherique(pa)",
                      "Meilleure destination en fonction de la probabilité de pleuvoir"],
            
            'colorbar': ['balance', 'oranges', 'blugrn', 'ice', 'agsunset', 'blues', 'aggrnyl'],
            
            'visible' : [True, False, False, False, False, False, False]
            
            }

    return dicts

In [26]:
dicts = create_dicts(new_df, top=20)

dict_df = pd.DataFrame(dicts)
dict_df

,df,df_normalized,criterion,title,colorbar,visible
0,city_id name latitude lon...,city_id name latitude lon...,score,Meilleur destination en fonction du score global,balance,True
1,city_id name latitu...,city_id name latitu...,mean_temp,Meilleur destination en fonction de la tempert...,oranges,False
2,city_id name latitu...,city_id name latitu...,mean_humidity,Meilleur destination en fonction du taux d'hum...,blugrn,False
3,city_id name la...,city_id name la...,mean_wind_speed,Meilleur destination en fonction de la vitesse...,ice,False
4,city_id name latitu...,city_id name latitu...,mode_weather_num,Meilleure destination en fonction du ciel,agsunset,False
5,city_id name la...,city_id name la...,mean_pressure,Meilleure destination en fonction de la pressi...,blues,False
6,city_id name latitu...,city_id name latitu...,mean_rain,Meilleure destination en fonction de la probab...,aggrnyl,False


In [27]:
def best_destination_map(df, top, mapbox_style, earth=False, normalized=False, dict_=None):


    token = 'pk.eyJ1Ijoib3AzbjVlZCIsImEiOiJjbDllYjl6bGswaG9uM3NsOW0zaGJ4ZHVrIn0.Us-gSPz0QgMbKbPqGkDtjg'
    cols = ['score', 'mean_temp', 'mean_rain', 'mean_humidity', 'mean_wind_speed', 'mean_pressure', 'mode_weather_num']

    dict_df = pd.DataFrame(dicts)

    fig = go.Figure()

    for i in range(dict_df.shape[0]):
      if normalized :
        fig.add_trace(
            go.Scattermapbox(
                lat=dict_df.loc[i, 'df']['latitude'],
                lon=dict_df.loc[i, 'df']['longitude'],
                hovertext=dict_df.loc[i, 'df']['name'],
                hoverinfo='text',
#                hovertemplate='<br>'.join([
#                f"Temperature: {dict_df.loc[i, 'data']['mean_temp']}",
#                'Humidité: {humidity}',
#                'Probabilité de pluie: {rain}',
#                ]),
                mode='markers',
                marker=dict(
                    size=dict_df.loc[i, 'df_normalized'][dicts['criterion'][i]]*65,
                    cmax=dict_df.loc[i, 'df'][dicts['criterion'][i]].max(),
                    cmin=dict_df.loc[i, 'df'][dicts['criterion'][i]].min(),
                    color=dict_df.loc[i, 'df'][dicts['criterion'][i]],
                    colorscale=dict_df.loc[i, 'colorbar'],
                    colorbar=dict(
                        title=dict(
                            side='right',
                            text=dict_df.loc[i, 'title'],
                            font=dict(
                                color='black',
                                size=15,
                                family='Arial'
                                )
                            ),
                    bgcolor='LightSkyBlue',
                    x=1.08,
                    y=0.5,
                    len=1.1)
                    ),
                visible=dict_df.loc[i, 'visible']
                )
        )
      else:
        fig.add_trace(
            go.Scattermapbox(
                lat=dict_df.loc[i, 'df']['latitude'],
                lon=dict_df.loc[i, 'df']['longitude'],
                hovertext=dict_df.loc[i, 'df']['name'],
                hoverinfo='text',
                hovertemplate='<br>'.join([
                f'Temperature: {df}',
                'Humidité: %{x}',
                'Probabilité de pluie: %{customdata}',
                ]),
                mode='markers',
                marker=dict(
                    size=32,
                    cmax=dict_df.loc[i, 'df'][dicts['criterion'][i]].max(),
                    cmin=dict_df.loc[i, 'df'][dicts['criterion'][i]].min(),
                    color=dict_df.loc[i, 'df'][dicts['criterion'][i]],
                    colorscale=dict_df.loc[i, 'colorbar'],
                    colorbar=dict(
                        title=dict(
                            side='right',
                            text=dict_df.loc[i, 'title'],
                            font=dict(
                                color='black',
                                size=15,
                                family='Arial'
                                )
                            ),
                    bgcolor='LightSkyBlue',
                    x=1.08,
                    y=0.5,
                    len=1.1)
                    ),
                visible=dict_df.loc[i, 'visible']
                )
        )
        

    start = df['date'][0].strftime("%m/%d/%Y")
    end = df['end_date'][0].strftime("%m/%d/%Y")

    fig.update_layout(mapbox_style=mapbox_style, 
                      mapbox_accesstoken=token,
                     title=dict(
                        text=f'Top {top} des villes en fonctions des conditions météo du {start} au {end}',
                        font=dict(
                            color='rgb(47, 138, 196)',
                            size=32,
                            family='Open Sans'
                        )
                    ))


    fig.update_mapboxes(
        bearing=0,
        center=dict(
            lat=46,
            lon=4
        ),
        pitch=0,
        zoom=3.6)

    fig.update_layout(
        updatemenus = [go.layout.Updatemenu(
            active = 0,
            bgcolor = "#4BE8E0",
            bordercolor = "#4B9AC7",
            buttons = [
                    go.layout.updatemenu.Button(
                        label = "Conditions générales météorologiques",
                        method = "update",
                        args = [{"visible" : [True, False, False, False, False, False, False]}]),
                    go.layout.updatemenu.Button(
                            label = "Moyenne des températures ",
                            method = "update",
                            args = [{"visible" : [False, True, False, False, False, False, False]}]),
                    go.layout.updatemenu.Button(
                            label = "Moyenne de la vitesse du vent",
                            method = "update",
                            args = [{"visible" : [False, False, True, False, False, False, False]}]),
                    go.layout.updatemenu.Button(
                            label = "Moyenne du taux d'humidité",
                            method = "update",
                            args = [{"visible" : [False, False, False, True, False, False, False]}]),
                    go.layout.updatemenu.Button(
                            label = "Visibilité du ciel",
                            method = "update",
                            args = [{"visible" : [False, False, False, False, True, False, False]}]),
                    go.layout.updatemenu.Button(
                            label = "Moyenne de pression atmospherique (Pa)",
                            method = "update",
                            args = [{"visible" : [False, False, False, False, False, True, False]}]),
                    go.layout.updatemenu.Button(
                            label = "Moyenne de probabilité de pluie",
                            method = "update",
                            args = [{"visible" : [False, False, False, False, False, False, True]}])
                ]
            )]
        )
    if earth:
      fig.update_layout(
      mapbox_style="white-bg",
      mapbox_layers=[
          {
              "below": 'traces',
              "sourcetype": "raster",
              "sourceattribution": "United States Geological Survey",
              "source": [
                  "https://basemap.nationalmap.gov/arcgis/rest/services/USGSImageryOnly/MapServer/tile/{z}/{y}/{x}"
              ]
          }
        ])
      fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

    fig.show()

In [28]:
best_destination_map(new_df,top=20, mapbox_style="open-street-map", earth=False, normalized=True, dict_=dicts)

### Hotels

Pour les hotels, nous allons créer 5 maps qui représente chacune une des 5 meilleurs destinations avec les meilleurs hotels associés et nous allons afficher en dessous de chaque carte une liste de ces meilleurs hotels avec différents critères

In [29]:
def format_best_hotels(df_hotels, df_weather, best_cities):
    df_hotels.dropna(inplace=True)
    df_hotels['description'] = df_hotels['description'].apply(lambda x:x.replace('\xa0', ' '))
    best_hotels = pd.DataFrame()

    for city in best_cities:
        globals()[f"best_{city.lower().replace('-', '_').replace(' ','_')}"] = df_hotels[df_hotels['Ville']==city].sort_values(by='score', ascending=False)[:20]
        globals()[f"best_{city.lower().replace('-', '_').replace(' ','_')}"]['rank'] = [i for i in range(1,21)]
        best_hotels = pd.concat([best_hotels, globals()[f"best_{city.lower().replace('-', '_').replace(' ','_')}"]], axis=0)
    
    l = [[df_weather['mean_temp'].iloc[i]]*20 for i in range(5)]
    temp = [c for el in l for c in el]

    best_hotels['temperature'] = temp

    scores = best_hotels['score'].apply(lambda x:str(x).replace(',','.'))

    best_hotels['score'] = scores.apply(lambda x:float(x))

    return best_hotels

In [30]:
best_five_cities = ['Ariege', 'Chateau-du-Haut-Koenigsbourg', 'Lille', 'Dijon', 'Annecy']

best_hotels = format_best_hotels(df_hotels, df_weather, best_five_cities)

In [31]:
def best_hotels_maps(best_hotels, best_cities):
    for city in best_cities:
        new_df = best_hotels[best_hotels['Ville']==city].reset_index()
      
        fig = px.scatter_mapbox(
            best_hotels[best_hotels['Ville'] == city],
            lat='lat',
            lon='lon',
            color='rank',
            title=f"{city} : {best_hotels[best_hotels['Ville'] == city]['temperature'].mean()}°C",
            size='score',
            color_continuous_scale=px.colors.diverging.RdYlGn,
            size_max=15,
            zoom=12,
            labels={'score': 'Score', 'location': 'Location'},
            mapbox_style="carto-positron")
          
        fig.show(renderer="colab")
      
        display(best_hotels[best_hotels['Ville']==city][['title','location','description', 'review_count', 'score','rank']].set_index('rank'))

In [32]:
best_hotels_maps(best_hotels, best_five_cities)

,title,location,description,review_count,score
rank,,,,,
1,dtente sky lodge,Massat,Hébergement géré par un particulier,93 experiences vecues,9.4
2,"Demeure de la Carmine, sejour bien-etre d'exce...",Mirepoix,"Située à Moulis, à seulement 32 km du col de l...",153 experiences vecues,9.4
3,la maison d'Anna,Ganac,"Doté d'un restaurant, d'un bar et offrant une ...",89 experiences vecues,9.4
4,Chambres d'hotes Belle Occitane,Tarascon-sur-Ariège,L'Auberge Des Deux Rivières est située en plei...,409 experiences vecues,9.2
5,Relais Mira Peis,Mirepoix,"Situé à Vicdessos, en Midi-Pyrénées, l'établis...",46 experiences vecues,9.1
6,Chambres d'Hotes La Maison Blanche,Lorp Sentaraille,"Situé à Biert, en Occitanie, à 47 km d'Andorre...",21 experiences vecues,9.1
7,La Maison des Consuls,Mirepoix,"Situé à Bénac, le Pyrénées Hideaway propose un...",127 experiences vecues,8.9
8,Skymist Maison d'hotes,Biert,"Offrant une vue sur son jardin, l'établissemen...",125 experiences vecues,8.8
9,Le Hameau du Comte de Foix,Mercus-Garrabet,Hébergement géré par un particulier,486 experiences vecues,8.7


,title,location,description,review_count,score
rank,,,,,
1,Villa Grimm,Sainte-Marie-aux-Mines,"Situé à Sélestat, l'établissement L'Appart' Sé...",118 experiences vecues,9.8
2,La Maison 1603,Hunawihr,Hébergement géré par un particulier,63 experiences vecues,9.5
3,La Maison de Vacances,Nothalten,Hébergement géré par un particulier,16 experiences vecues,9.5
4,Villa Maeva - Logements,Riquewihr,"Situé dans la région viticole du Bas-Rhin, le ...",53 experiences vecues,9.5
5,Laterale Residences Apartments Riquewihr,Riquewihr,"Situé à Bergheim, en Alsace, à 14 km de Colmar...",144 experiences vecues,9.4
6,A l'ombre du verger,Breitenau,L’établissement LE CLOS DES SOURCES Hôtel & Sp...,77 experiences vecues,9.4
7,Gite L'Oree du Chateau - Appartement Potiron,Orschwiller,Hébergement géré par un particulier,6 experiences vecues,9.3
8,Gite Binnert Bader,Nothalten,"Situé à Hunawihr, à 13 km du parc des expositi...",19 experiences vecues,9.3
9,Gite L oree du chateau Appartement Cerise,Orschwiller,"Offrant une vue sur la montagne, le Gîte L'Oré...",6 experiences vecues,9.2


,title,location,description,review_count,score
rank,,,,,
1,L'Esplanade Lille,"Vieux-Lille, Lille",L'établissement L'Esplanade Lille est situé da...,816 experiences vecues,9.2
2,Aparthotel Adagio Lille Centre Grand Place,"Centre de Lille, Lille",Doté d'un salon commun et d'une connexion Wi-F...,1 850 experiences vecues,8.9
3,Hotel Barriere Lille,"Centre de Lille, Lille","Établissement 5 étoiles, l'Hôtel Barrière Lill...",1 062 experiences vecues,8.7
4,OKKO Hotels Lille Centre,"Centre de Lille, Lille","Situé à Lille, l’OKKO Hotels Lille Centre prop...",1 320 experiences vecues,8.7
5,Moxy Lille City,"Centre de Lille, Lille","Situé à Lille, à 300 mètres de la maison Coill...",3 003 experiences vecues,8.6
6,Grand Hotel Bellevue - Grand Place,"Centre de Lille, Lille","Situé sur la Grand-Place, dans le Vieux-Lille,...",2 801 experiences vecues,8.4
7,Lille A Part - Gites Urbains,"Vieux-Lille, Lille","Situé au cœur de la vieille ville de Lille, le...",166 experiences vecues,8.2
8,"L'Hermitage Gantois, Autograph Collection","Centre de Lille, Lille","Doté d'un spa sur place, l'établissement 5 éto...",756 experiences vecues,8.2
9,Campanile Lille Euralille,Lille,Le Campanile Lille Euralille vous accueille à ...,3 081 experiences vecues,8.1


,title,location,description,review_count,score
rank,,,,,
1,VILLA14,"Centre-ville de Dijon, Dijon","Située dans le centre de Dijon, la VILLA14 se ...",307 experiences vecues,9.0
2,Chambre d'hotes Le Petit Tertre,"Centre-ville de Dijon, Dijon",L'établissement le Petit Tertre est situé dans...,701 experiences vecues,8.9
3,Hotel des Halles,"Centre-ville de Dijon, Dijon",L'Hôtel des Halles vous accueille dans le cent...,3 447 experiences vecues,8.8
4,Hotel des Ducs,"Centre-ville de Dijon, Dijon","Situé à 50 mètres du palais des ducs, dans le ...",1 811 experiences vecues,8.5
5,Oceania Le Jura Dijon,"Centre-ville de Dijon, Dijon",L'Oceania Le Jura Dijon vous accueille dans le...,1 638 experiences vecues,8.4
6,Kyriad Prestige Dijon Nord - Valmy,Dijon,"Disposant d'une réception ouverte 24h/24, le K...",2 120 experiences vecues,8.3
7,Aparthotel Adagio Access Dijon Republique,Dijon,"Situé dans le centre-ville de Dijon, à seuleme...",1 784 experiences vecues,8.3
8,Vertigo | a Member of Design Hotels,"Centre-ville de Dijon, Dijon","Proposant une piscine intérieure, un spa Nuxe ...",1 562 experiences vecues,8.1
9,City Loft Apparthotel,"Centre-ville de Dijon, Dijon",Idéalement situé au cœur du quartier piétonnie...,3 340 experiences vecues,8.1


,title,location,description,review_count,score
rank,,,,,
1,L'appartement d'Anna,"Centre-ville d'Annecy, Annecy","Situé à Annecy, à 35 km de Rochexpo et à 36 km...",103 experiences vecues,9.2
2,Rivage Hotel & Spa Annecy,Annecy,L'Hôtel Le Pré Carré est situé au cœur d'Annec...,1 782 experiences vecues,8.9
3,Allobroges Park Hotel,"Centre-ville d'Annecy, Annecy",Installé juste à proximité des rives du lac d’...,1 998 experiences vecues,8.7
4,Majord'Home Spa 5* - La Source Coeur Vieille V...,"Centre-ville d'Annecy, Annecy","Doté d'une baignoire spa, le Majord'Home Spa 5...",18 experiences vecues,8.7
5,Le Pelican,Annecy,"Situé dans le centre d'Annecy, l'Hôtel du Nord...",998 experiences vecues,8.6
6,Apartment near the lake the castle and the cit...,Annecy,L'hôtel Le Bel Abri occupe une ancienne maison...,5 experiences vecues,8.6
7,LE MIRAGE - Annecy Centre - Appartement de lux...,"Centre-ville d'Annecy, Annecy","Situé dans le centre d'Annecy, à seulement 35 ...",57 experiences vecues,8.6
8,Le Bel Abri,Annecy,Situé à 500 mètres du lac d'Annecy et à 12 min...,589 experiences vecues,8.6
9,Hotel du Chateau,"Centre-ville d'Annecy, Annecy","Offrant une vue sur la vieille ville, l’Hotel ...",1 540 experiences vecues,8.5
